In [86]:
292701

292701

In [87]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Tarefa 3 - Neural Networks
Third assessed coursework for the course: Técnicas e Algoritmos em Ciência de Dados

This tarefa provides an exciting opportunity for students to put their knowledge acquired in class into practice, using neural networks to solve real-world problems in both classification and regression. Students will apply the concepts they have learned to build, train, and optimize neural networks, using a validation set to fine-tune hyperparameters. Students will also get used to generating important plots during training to analyse the models' behaviour. By the end of the project, students will have gained hands-on experience in implementing neural networks.

## General guidelines:

* This work must be entirely original. You are allowed to research documentation for specific libraries, but copying solutions from the internet or your classmates is strictly prohibited. Any such actions will result in a deduction of points for the coursework.
* Please enter your code in the designated areas of the notebook. You can create additional code cells to experiment with, but __make sure to place your final solutions where they are requested in the notebook.__
* Before submitting your work, make sure to rename the file to the random number that you created for the previous coursework (for example, 289479.ipynb).

## Notebook Overview:

1. [Regression](#Regression) (50%)
2. [Classification](#Classification) (50%)

# Regression

**Download from ECLASS**
- Tarefa_3_template.ipynb
- energy_efficiency.csv

**Dataset and Problem Description**

In this exercise, you will use the Energy Efficiency Prediction dataset. This dataset contains information about the energy efficiency of buildings based on eight features, including the size of the building, the orientation, and the type of building materials used. The dataset includes two targets: heating load and cooling load, which represent the energy required to heat and cool the building, respectively.

This dataset is useful for building neural networks that predict the energy efficiency of buildings, which is an important problem in the field of sustainable energy. The dataset has been used in several machine learning research papers and provides a challenging regression problem.

**Exercise Description: Energy Efficiency Prediction with Neural Networks**

In this exercise, you will use the Energy Efficiency Prediction dataset provided.
You will build and train a neural network to predict the heating load (column labelled y1 in the dataset) and the cooling load (column labelled y2) of a building based on its energy efficiency features. 


### To complete this exercise, you will write code to build and train neural networks for this problem:

1. Split the dataset into training, validation, and test sets, using a 70:15:15 ratio.

2. Use numpy, build a neural network that takes the energy efficiency features as input and predicts the heating and the cooling load as outputs. You will choose the number of neurons per layers and the number of layers, but each layer will have the same number of neurons. These two values must be input parameters for your neural network. That is, you can’t hard-code each layer, meaning that you will have to write code that is able to work with a variable number of layers and neurons. 

3. Code the forward pass and backpropagation algorithm to learn the weights of the neural network. Use the training set to train the neural network and update the weights using stochastic gradient descent. For the hidden layers use the sigmoid activation function. You will need to regularize your neural network using weight decay, that is, you will include a regularization term in your error function.

4. Monitor the training by plotting the training and validation losses across the epochs. 

The performance of your neural network will be different depending on the number of layers, number of neurons per layer and the value of λ that controls the amount of weight decay. You will experiment with 3 values of λ: 0 (no weight decay), 0.001 and 0.0001.
To choose the best network configuration and assess its performance you will:

1. Choose 3 possible values of number hidden layers (for example, 1 to 3) and 3 different values of neurons per layer (for example, 100, 200, and 300), but you can also choose different values. 

2. Calculate the loss for each configuration on the validation set.

3. Generate 3 heatmaps, one for each value of the λ regularization parameter, displaying the loss on the validation set by plotting the number of layers and number of neurons in a grid. This will help you visualise the best configuration for the neural network. 

4. Train your final model selecting the best combination of hyper-parameters and evaluate the final performance of the neural network using the test set and the root mean squared error as the metric and report that.

**Important:**
- Train for 50 epochs.
- Set the learning rate η to 0.01.


In [88]:
# Reading csv and separating between data and target
data = pd.read_csv("./energy_efficiency.csv")
X = data.values[:,:-2]
y = data.values[:,-2:]
# Spliting the data
X_train, X_tv, y_train, y_tv = train_test_split(X, y, test_size=0.3, random_state=47)
X_test, X_val, y_test, y_val = train_test_split(X_tv, y_tv, test_size=0.5, random_state=47)

In [93]:
## your code goes here:
def rmse(y_pred:np.ndarray, y_real:np.ndarray) -> float:
    return np.sqrt(np.mean(np.power(y_pred - y_real, 2)))

def sse(y_pred:np.ndarray, y_real:np.ndarray) -> float:
    return np.sum(np.power(y_pred - y_real, 2))

def sigmoid_activation(z:np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z:np.ndarray) -> np.ndarray:
    return sigmoid_activation(z) * (1 - sigmoid_activation(z))

def gen_weights_bias(nlines: int, ncols: int, std:float=0.0001) -> list[np.ndarray]:
    return [np.random.normal(size=(nlines, ncols), scale=std), np.random.normal(size=(ncols, 1), scale=std)]

def get_rmse_model(X: np.ndarray, y_real:np.ndarray, model:dict, activation=sigmoid_activation) -> float:
    X = X @ model[0][0] * model[0][1].T

    for layer in list(model.keys())[1:]:
        X = activation(X) @ model[layer][0] * model[layer][1].T

    y_pred = X
    return rmse(y_pred=y_pred, y_real=y_real)

def neural_network_train(X:np.ndarray, y_real:np.ndarray, neurons_amount:int,
                         layers_num:int, epochs:int,
                         weight_decay:float, learning_rate:float) -> list[list[np.ndarray]]:
    # Initialize the layers with random values and save in a list of layers
    model_dict = dict()


    if layers_num == 1:
        model_dict[0] = gen_weights_bias(X.shape[1], y_real.shape[1])
    else:
        model_dict[0] = gen_weights_bias(X.shape[1], neurons_amount)

        if layers_num > 2:
            for i in range(1, layers_num-1):
                model_dict[i] = gen_weights_bias(neurons_amount, neurons_amount)
                
        model_dict[layers_num-1] = gen_weights_bias(neurons_amount, y_real.shape[1])

    sse_historic = list()
    # For each epoch we ran our train again
    for epoch in range(epochs):
        # Permutate the indices to exclude a vies
        indices = np.random.permutation(len(X))
        X_permuted = X[indices]
        y_permuted = y_real[indices]

        # Start the Stochastic Gradient Descendent method
        for i in range(len(X)):
            # Get a sample of X and y permuted
            x_sample = X_permuted[i]
            y_sample = y_permuted[i]
            
            # X sample is a line vector
            x_sample = x_sample.reshape(1, -1)
            y_sample = y_sample.reshape(1, -1)
            
            # Start the layers activations by doing
            layers = dict()
            
            layers[0] = x_sample @ model_dict[0][0] + model_dict[0][1].T
            
            for i in range(1, layers_num):
                layers[i] = sigmoid_activation(layers[i-1]) @ model_dict[i][0] + model_dict[i][1].T
                
            error_current = layers[layers_num-1] - y_sample # e_o = a_o - y

            for i in range(layers_num-1, 0, -1):

                derivative_w = sigmoid_activation(layers[i-1]).T @ error_current # DW_o = a_o-1.T @ e_o
                derivative_b = np.sum(error_current, axis=0).reshape(-1, 1) # Db_o = e_o
                
                model_dict[i][0] -= learning_rate * (derivative_w + weight_decay * model_dict[i][0])
                model_dict[i][1] -= learning_rate * derivative_b

                error_current = (error_current @ model_dict[i][0].T) * sigmoid_derivative(layers[i-1])

            derivative_wi = x_sample.T @ error_current # DW_o = a_o-1.T @ e_o
            derivative_bi = np.sum(error_current, axis=0).reshape(-1, 1) # Db_o = e_o
                
            model_dict[0][0] -= learning_rate * (derivative_wi + weight_decay * model_dict[0][0])
            model_dict[0][1] -= learning_rate * derivative_bi

        sse_current = get_rmse_model(X, y_real, model_dict)
        sse_historic.append(sse_current)

    return model_dict, sse_historic

neural_network_train(X_train, y_train, 100, 3, 50, 0.01, 0.01)[1]

[0.47377713055173376,
 0.4734677351086591,
 0.47562775150048936,
 0.47058991074914397,
 0.4721836996291171,
 0.4652288393665641,
 0.46853408245046735,
 0.46973378959798384,
 0.4642447436955236,
 0.47428844752445715,
 0.46478336088248445,
 0.4687646369945157,
 0.4696583290705276,
 0.4600429627045762,
 0.4664292534385199,
 0.47011226349357477,
 0.4666319507314631,
 0.47037101217636107,
 0.47105022421179715,
 0.4710362977094078,
 0.4682904699170062,
 0.4701382085488013,
 0.465144206406122,
 0.4742375051734222,
 0.4705133190591716,
 0.46998539056069794,
 0.4631933001858971,
 0.46439615486193697,
 0.46874578158481806,
 0.46892133568845196,
 0.46801950905502615,
 0.4730849581566854,
 0.46546236812371405,
 0.4688666416674771,
 0.4588510505035938,
 0.4652858975301485,
 0.46557583778829326,
 0.4669341927343519,
 0.4606712921907032,
 0.46694298086956687,
 0.4595502724021547,
 0.46903761605323246,
 0.4634047008732964,
 0.46580836649332646,
 0.4676696979027278,
 0.4657980686892097,
 0.467889967951

# Classification

**Download the data from ECLASS**
- drug_side_effects.csv
- drug_features.csv

**Dataset description:**

In this exercise, you will build and train a neural network to predict the occurrence of drug side effects. The dataset is derived from the SIDER dataset, containing relatively common side effects that can occur for at least 50 drugs. This produces a total of 740 drugs and 256 side effects. The features represent various molecular properties, including molecular weight, number of atoms, number of rings, number of hydrogen bond donors and acceptors, logP, topological polar surface area (TPSA), number of rotatable bonds, number of aromatic rings, number of aliphatic rings, number of saturated rings, and number of heteroatoms. 

**Remember that each drug can cause many side effects, not only one.** 

*Feel free to explore the dataset and check the potential side effects of popular medications!*

### To complete this exercise, follow these steps:

1. Load the dataset and split it into training, validation, and test sets, using an 80:10:10 ratio. 

2. Standardize the features by removing the mean and scaling to unit variance. To do this, perform the following for each feature (column) in the dataset:
    - Calculate the mean and standard deviation across the training set for that feature.
    - Subtract the mean from each value in that feature and divide by the standard deviation.
    - Apply the same transformation to the validation and test sets using the mean and standard deviation calculated from the training set.

**Observation:** you need to code this part, you’re not allowed to use scikit-learn.

*Normalization of features is important for neural networks because:*
- *It ensures that all features have the same scale, preventing certain features from dominating the learning process due to their larger magnitude.*
- *It improves the numerical stability of the training process, making the neural network less sensitive to the choice of learning rate and other hyperparameters.*

3. Build a neural network using NumPy that takes in the features as input and predicts the occurrence of side effects. You will choose the number of neurons per layer and the number of layers. You will provide this information as an input list where the length of the list determines the number of hidden layers, and each element is the number of neurons of that hidden layer. For example, an array `layers = [64,128,256]` should produce a network with 4 layers, with 3 hidden layers with 64, 128, and 256 neurons each. For the hidden layers use the sigmoid activation function. You will need to regularize your neural network using weight decay, that is, you will include a regularization term in your error function.

4. Code the forward pass and backpropagation algorithm to learn the weights of the neural network. Use the training set to train the neural network and update the weights using stochastic gradient descent. Don’t forget about the biases. 

5. Monitor the training by plotting the training and validation losses across the epochs.

	**Observation:** make sure the loss goes down during training, acceptable values are within 0.2 – 2.8 approximately. These values depend on the choice of the different hyperparameters. Test only sensible values taking into account the dataset, i.e., number of features, drugs, side effects. 

The performance of your neural network will be different depending on the number of layers, number of neurons per layer and the value of λ that controls the amount of weight decay. You will experiment with 3 values of λ: 0 (no weight decay), 1 and 0.01.
To choose the best network configuration and assess its performance you will:

1. For each value of λ, select 3 different layer configurations (note that in this exercise, the number of neurons per layer does not require to be the same for each layer).
2. Calculate the loss for each configuration on the validation set.
3. At the end of this process, you should be left with 9 loss values (one for each configuration). Train your final model selecting the best combination of hyper-parameters and evaluate the final performance of the neural network using the test set and the Area Under the ROC Curve (AUROC) with the function provided in the Jupyter notebook. 
	
	*Observation: don’t expect impressive AUROC values, as this is a highly complex problem that can’t be solved easily with a simple neural network with standard features. Expect values in the range (0.55-0.75).*

**Important:**
- Train for 50 epochs.
- Set the learning rate η to 0.01.



In [ ]:
## your code goes here: 

In [ ]:
## to calculate the test set AUROC use the following code:

auroc = roc_auc_score()

TypeError: missing a required argument: 'y_true'